<a href="https://colab.research.google.com/github/maaniaxs/Deep-Learning/blob/main/DL_NLP_with_GRU%2CConv1D%2ClSTM_21_11_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-771c2c68-3161-8dbf-2aa7-54779bc23dcd)


In [2]:
import pandas as pd

In [3]:
import zipfile

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/nlp_getting_started.zip', "r")
zip_ref.extractall()
zip_ref.close()

In [4]:
train = pd.read_csv('/content/train.csv')
print(train.shape)
train.head()

(7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [6]:
import nltk
import re

## DATASET PREPROCESSING
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
preprocess_text = []
for text in train['text']:
    #print(i)
    review= re.sub(r'http\S+', ' ', text)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review= re.sub(r'http\S+', ' ', review)
    review = review.lower()
    review = re.sub('\s+',' ', review)
    review = review.split()
    
    review = [ps.stem(word) for word in review]
    review = ' '.join(review)
    preprocess_text.append(review)

In [7]:
print(len(preprocess_text))

train['clean_text'] = preprocess_text

7613


In [103]:
train.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deed are the reason of thi earthquak may a...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resid ask to shelter in place are be notif...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order in california
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent thi photo from rubi alaska as sm...


In [8]:
# shuffle training data
train_df = train.sample(frac=1, random_state=9)
train_df.head()

,id,keyword,location,text,target,clean_text
4006,5689,floods,Philippines,PAGASA 7:12am: YELLOW warning - Panay Island G...,1,pagasa am yellow warn panay island guimara neg...
2574,3691,destroy,NaN,Save your riches in heaven where they will nev...,0,save your rich in heaven where they will never...
1345,1943,burning%20buildings,"New Orleans ,Louisiana",Burning buildings? Media outrage? http://t.co/...,1,burn build media outrag
4526,6434,hurricane,NaN,Ashley and I on going to hurricane harbor Frid...,1,ashley and i on go to hurrican harbor friday
2295,3292,demolish,NaN,I have completed the quest 'Demolish 5 Murlo.....,0,i have complet the quest demolish murlo in the...


In [10]:

## SPLIT DATASET INTO TRAINING AND VALIDATION SET
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_df['clean_text'].to_numpy(),
                                                                           train_df['target'].to_numpy(),
                                                                           test_size=0.1, random_state=59)

In [11]:
length = []
for i in x_train:
  length.append(len(i.split()))
total = 0
for i in range(0, len(length)):
  total = total+ length[i]
print(total)

97582


In [12]:
##  FOR EXAMPLE PURPOSE
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
max_vocab_length = 12500 # maximum number of words to have in our vocabulary
max_length= 10  # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens= max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length= max_length)  # each sentence/rows have length 40

text_vectorizer.adapt(x_train)

vocab_word = text_vectorizer.get_vocabulary() # get all the unique words from our training data
print(len(vocab_word))                                 

12378


In [76]:
text_vectorizer([x_train[0]])

<tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[9132,    7,  106,  101,   80,   18,  940, 4883,    0,    0]])>

In [75]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim= max_vocab_length,  # total unique word in vocab
                             output_dim = 5,
                             input_length= max_length)  # how long is each input

In [77]:
embedding(text_vectorizer([x_train[0]]))

<tf.Tensor: shape=(1, 10, 5), dtype=float32, numpy=
array([[[-0.04233098,  0.00958087, -0.00054305, -0.03273143,
         -0.04638107],
        [ 0.01337106,  0.03877056,  0.01018842,  0.00706784,
          0.02892491],
        [-0.00377252, -0.0154649 ,  0.0180533 , -0.01173962,
         -0.00289148],
        [ 0.005665  , -0.03382754, -0.00183278,  0.04386455,
          0.04647653],
        [ 0.04644886,  0.01683087,  0.02107463, -0.03872876,
         -0.01559917],
        [ 0.04752732,  0.04533186, -0.00446985, -0.03416703,
         -0.00658041],
        [-0.0217394 , -0.03027772, -0.00704242, -0.01765757,
         -0.02816783],
        [ 0.00456624,  0.00782444, -0.02734802,  0.00179446,
          0.02506064],
        [-0.01675023, -0.04339694,  0.01792837, -0.04441161,
          0.04874155],
        [-0.01675023, -0.04339694,  0.01792837, -0.04441161,
          0.04874155]]], dtype=float32)>

# ************

 ## 1 Building model using Tensorflow

In [13]:
## text vectorization
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#max_vocab_length = 12500 # maximum number of words to have in our vocabulary
max_length= 35  # max length our sequences will be

vectorizer = TextVectorization(max_tokens= None,
                                    output_mode='int',
                                    output_sequence_length= max_length)  # each sentence/rows have length 40

vectorizer.adapt(x_train)

vocab_word = vectorizer.get_vocabulary() # get all the unique words from our training data
print(len(vocab_word))

12378


In [14]:
from tensorflow.keras import layers

Embedding = layers.Embedding(input_dim= 12378,  # total unique word in vocab
                             output_dim = 24,
                             input_length= max_length)  # how long is each input

In [15]:
#                       1
# create as LSTM model
import tensorflow as tf
from tensorflow.keras import layers
inputs = layers.Input(shape= (1,), dtype= 'string')
x= vectorizer(inputs)
x= Embedding(x)
print(x.shape)
x= layers.LSTM(128, activation='tanh')(x)  # if return_sequence=True then we add one LSTM layer with (return_sequences=False)
#print(x.shape)
#x= layers.LSTM(24, activation= 'LeakyRelu')(x)       ## GRU & 1D Pooling layer can also be used instead 
print(x.shape)
x= layers.Dense(64, activation='relu')(x)
print(x.shape)
outputs= layers.Dense(1, activation='sigmoid')(x)
print(outputs.shape)
model_2 = tf.keras.Model(inputs, outputs, name= 'model_2_LSTM')

# compile model
model_2.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])

model_2.summary()

(None, 35, 24)
(None, 128)
(None, 64)
(None, 1)
Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 35)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 35, 24)            297072    
                                                                 
 lstm (LSTM)                 (None, 128)               78336     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
      

In [16]:
model_2.fit(x_train, y_train, epochs=5, validation_data= (x_test, y_test))

Epoch 1/5
215/215 [==============================] - 8s 10ms/step - loss: 0.6169 - accuracy: 0.6548 - val_loss: 0.5125 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3615 - accuracy: 0.8577 - val_loss: 0.6674 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2394 - accuracy: 0.9127 - val_loss: 0.5194 - val_accuracy: 0.7428
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1758 - accuracy: 0.9442 - val_loss: 0.7343 - val_accuracy: 0.7310
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1395 - accuracy: 0.9568 - val_loss: 0.7719 - val_accuracy: 0.7310


In [18]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [19]:
## Evaluating and make prediction with our model
model_pred  = model_2.predict(x_test)

model_pred = tf.squeeze(tf.round(model_pred))
# calculate model result
model_results= calculate_results(y_test, model_pred)
model_results

24/24 [==============================] - 0s 3ms/step


{'accuracy': 73.0971128608924,
 'precision': 0.7313749751551716,
 'recall': 0.7309711286089239,
 'f1': 0.7285621269606322}

In [34]:
#                       2nd Model Building with GRU layers
#
import tensorflow as tf
from tensorflow.keras import layers
inputs = layers.Input(shape= (1,), dtype= 'string')
x= vectorizer(inputs)
x= Embedding(x)
print(x.shape)
x = layers.GRU(72, return_sequences=True)(x)
print(x.shape)
x = layers.GlobalMaxPool1D()(x)                        # GRU with (return_sequences=Fasle) layers can also be used instead
x= layers.Dense(24, activation='relu')(x)
print(x.shape)
outputs= layers.Dense(1, activation='sigmoid')(x)
print(outputs.shape)
model_3 = tf.keras.Model(inputs, outputs)

# compile model
model_3.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])
model_3.summary()  # Model summary

(None, 35, 24)
(None, 35, 72)
(None, 24)
(None, 1)
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 35)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 35, 24)            297072    
                                                                 
 gru_7 (GRU)                 (None, 35, 72)            21168     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 72)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_1

In [35]:
# fit the model
model_3.fit(x_train, y_train, epochs=10, validation_data= (x_test, y_test))

Epoch 1/10
215/215 [==============================] - 4s 9ms/step - loss: 0.1711 - accuracy: 0.9571 - val_loss: 0.9989 - val_accuracy: 0.7113
Epoch 2/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0498 - accuracy: 0.9818 - val_loss: 1.1853 - val_accuracy: 0.6929
Epoch 3/10
215/215 [==============================] - 1s 7ms/step - loss: 0.0423 - accuracy: 0.9825 - val_loss: 1.3316 - val_accuracy: 0.7152
Epoch 4/10
215/215 [==============================] - 1s 7ms/step - loss: 0.0389 - accuracy: 0.9826 - val_loss: 1.2636 - val_accuracy: 0.7060
Epoch 5/10
215/215 [==============================] - 1s 7ms/step - loss: 0.0366 - accuracy: 0.9839 - val_loss: 1.6478 - val_accuracy: 0.6759
Epoch 6/10
215/215 [==============================] - 1s 7ms/step - loss: 0.0355 - accuracy: 0.9828 - val_loss: 1.4541 - val_accuracy: 0.6942
Epoch 7/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0349 - accuracy: 0.9844 - val_loss: 1.4060 - val_accuracy: 0.7047
Epoch 

In [38]:

 #                               Build a Conv1D model_5 
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x= vectorizer(inputs)
x = Embedding(x)
x = layers.Conv1D(filters=128, kernel_size=4, strides=1, padding='valid')(x)
#x = layers.Conv1D(filters=128, kernel_size=4)(x)

x = layers.GlobalAveragePooling1D()(x)
#x = layers.LSTM(64)(x)            # this layer can also be used instead of GlobalAveragePooling1D layer
x = layers.Dense(32, activation= 'relu')(x)
outputs = layers.Dense(1, activation= 'sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name = 'model_5_Conv1D')

# compile model_5
model_5.compile(loss= 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(), metrics=['accuracy'])
#
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 35)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 35, 24)            297072    
                                                                 
 conv1d_7 (Conv1D)           (None, 32, 128)           12416     
                                                                 
 global_average_pooling1d_2   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_16 (Dense)            (None, 32)             

In [39]:
# fit the model
model_5.fit(x_train, y_train, epochs=6, validation_data= (x_test, y_test))

Epoch 1/6
215/215 [==============================] - 2s 6ms/step - loss: 0.1347 - accuracy: 0.9715 - val_loss: 1.1788 - val_accuracy: 0.7073
Epoch 2/6
215/215 [==============================] - 1s 5ms/step - loss: 0.0493 - accuracy: 0.9804 - val_loss: 1.3078 - val_accuracy: 0.7218
Epoch 3/6
215/215 [==============================] - 1s 5ms/step - loss: 0.0422 - accuracy: 0.9828 - val_loss: 1.4368 - val_accuracy: 0.7139
Epoch 4/6
215/215 [==============================] - 1s 5ms/step - loss: 0.0399 - accuracy: 0.9823 - val_loss: 1.3849 - val_accuracy: 0.7100
Epoch 5/6
215/215 [==============================] - 1s 5ms/step - loss: 0.0386 - accuracy: 0.9829 - val_loss: 1.4526 - val_accuracy: 0.7073
Epoch 6/6
215/215 [==============================] - 1s 5ms/step - loss: 0.0369 - accuracy: 0.9835 - val_loss: 1.4196 - val_accuracy: 0.6929


In [ ]:
## If we do better text preprocessing then we could get good accuracy    

# #  BUILD TENSORFLOW MODEL TO ANOTHER WAY


In [190]:
train_df.reset_index(drop=True, inplace=True)
train_df.drop(['keyword', 'location'], axis=1, inplace=True)
train_df.head()

,id,text,target,clean_text
0,5689,PAGASA 7:12am: YELLOW warning - Panay Island G...,1,pagasa am yellow warn panay island guimara neg...
1,3691,Save your riches in heaven where they will nev...,0,save your rich in heaven where they will never...
2,1943,Burning buildings? Media outrage? http://t.co/...,1,burn build media outrag
3,6434,Ashley and I on going to hurricane harbor Frid...,1,ashley and i on go to hurrican harbor friday
4,3292,I have completed the quest 'Demolish 5 Murlo.....,0,i have complet the quest demolish murlo in the...


In [186]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [188]:

# ONE-HOT REPRESENTATION

voc_size= 13000  ## vocabulary-size  , means how many unique words are in there

onehot = [one_hot(words, voc_size) for words in train_df['clean_text']]
  # every rows of this text data convert into numeric form

In [191]:
onehot[0]

[4047,
 11706,
 3208,
 7034,
 8503,
 1383,
 3754,
 8884,
 1123,
 9231,
 4012,
 9194,
 8014,
 1717,
 8977,
 11880,
 4012,
 7094,
 1717]

In [192]:
train_df['clean_text'][0]

'pagasa am yellow warn panay island guimara negro possibl flood in low lie area amp landslid in mountain area'

In [193]:
## PADDING OF EACH SENTECES TO MAKE EQUAL LENGTH 
sent_length = 35  # length of each sentence
padded_doc = pad_sequences(onehot, padding='pre', maxlen=sent_length)

In [194]:
padded_doc[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,  4047, 11706,
        3208,  7034,  8503,  1383,  3754,  8884,  1123,  9231,  4012,
        9194,  8014,  1717,  8977, 11880,  4012,  7094,  1717],
      dtype=int32)

In [197]:
##
import numpy as np
print(len(padded_doc))
x_final = np.array(padded_doc)
y = train_df['target']
y_final = np.array(y)  # y = target
print(x_final.shape, y_final.shape)

from sklearn.model_selection import train_test_split  ## TRAIN TEST SPLIT
x_train, x_test, y_train,y_test = train_test_split(x_final,y_final, test_size=0.15, random_state=43)

7613
(7613, 35) (7613,)


In [204]:
## CREATING MODEL

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

#embed_vector_size = 32  # each words of every sentences convert into 30 no. of dense vector
model = Sequential()
# EMBEDDING
model.add(Embedding( voc_size, output_dim = 32,input_length = 35)) ## EMBEDDING LAYER
# model.add(Dropout(0.2))
model.add(LSTM(72))
# model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 35, 32)            416000    
                                                                 
 lstm_38 (LSTM)              (None, 72)                30240     
                                                                 
 dense_55 (Dense)            (None, 1)                 73        
                                                                 
Total params: 446,313
Trainable params: 446,313
Non-trainable params: 0
_________________________________________________________________


In [205]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
203/203 [==============================] - 9s 31ms/step - loss: 0.5590 - accuracy: 0.7073
Epoch 2/5
203/203 [==============================] - 6s 30ms/step - loss: 0.3601 - accuracy: 0.8481
Epoch 3/5
203/203 [==============================] - 6s 31ms/step - loss: 0.2641 - accuracy: 0.8946
Epoch 4/5
203/203 [==============================] - 6s 30ms/step - loss: 0.2016 - accuracy: 0.9271
Epoch 5/5
203/203 [==============================] - 6s 31ms/step - loss: 0.1527 - accuracy: 0.9450


In [206]:
model.evaluate(x_test, y_test)

36/36 [==============================] - 1s 9ms/step - loss: 0.6970 - accuracy: 0.7653


[0.6969825029373169, 0.7653239965438843]

In [208]:
#  BUILD TENSORFLOW MODEL TO ANOTHER WAY
from keras import Sequential
from keras.layers import Dense,LSTM, Embedding,GRU,  Flatten

model = Sequential()
model.add(Embedding(voc_size, 24, input_length= 35))
model.add(layers.GRU(128,return_sequences=True, activation = 'tanh'))
model.add(GRU(64))
model.add(Dense(32, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid'))

model.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 35, 24)            312000    
                                                                 
 gru_12 (GRU)                (None, 35, 128)           59136     
                                                                 
 gru_13 (GRU)                (None, 64)                37248     
                                                                 
 dense_56 (Dense)            (None, 32)                2080      
                                                                 
 dense_57 (Dense)            (None, 1)                 33        
                                                                 
Total params: 410,497
Trainable params: 410,497
Non-trainable params: 0
_________________________________________________________________


In [209]:
model.fit(x_train, y_train, epochs=5, validation_data= (x_test, y_test))

Epoch 1/5
203/203 [==============================] - 20s 74ms/step - loss: 0.5440 - accuracy: 0.7183 - val_loss: 0.4321 - val_accuracy: 0.8047
Epoch 2/5
203/203 [==============================] - 16s 80ms/step - loss: 0.3593 - accuracy: 0.8486 - val_loss: 0.4591 - val_accuracy: 0.8004
Epoch 3/5
203/203 [==============================] - 14s 70ms/step - loss: 0.2622 - accuracy: 0.8979 - val_loss: 0.5696 - val_accuracy: 0.7706
Epoch 4/5
203/203 [==============================] - 14s 70ms/step - loss: 0.2028 - accuracy: 0.9261 - val_loss: 0.6501 - val_accuracy: 0.7706
Epoch 5/5
203/203 [==============================] - 16s 77ms/step - loss: 0.1581 - accuracy: 0.9439 - val_loss: 0.6569 - val_accuracy: 0.7609


In [214]:
## Evaluating and make prediction with our model
model_pred  = model.predict(x_test)

model_pred = tf.squeeze(tf.round(model_pred))
# calculate model result
model_results= calculate_results(y_test, model_pred)
model_results

36/36 [==============================] - 1s 19ms/step


{'accuracy': 76.09457092819615,
 'precision': 0.7639355991438603,
 'recall': 0.7609457092819615,
 'f1': 0.7618791155004768}